# Build pointing data

This notebook reads data produced by `reducing_pointing_data` notebook and analyses it to produce corrected pointing data files.

## Parameterized notebook

This notebook is parameterized, which means it could be run with tools like Papermill as part of a data analysis pipeline.

In [ ]:
import os 
import pickle

import numpy as np
import matplotlib.pyplot as plt

from astropy import units as u
from datetime import datetime

from lsst.geom import PointD

from lsst.ts.observing.utilities.auxtel.latiss.utils import calculate_xy_offsets

from lsst.ts.observatory.control.constants.latiss_constants import boresight

## Notebook Parameters

The next cell define the notebook parameters

### Pickle file with the data to process

The notebooks needs the name of the pickle file containing the data to process.

In [ ]:
pointing_data_file = "data/20210609/AT_point_data_20210609_tw003.pickle"

## Processing the data

In [ ]:
with open(pointing_data_file, "rb") as fp:
    brightest_source_centroid = pickle.load(fp)

In [ ]:
def rotation_matrix(angle):
    """Rotation matrix.
    """
    return np.array(
        [
            [np.cos(np.radians(angle)), -np.sin(np.radians(angle)), 0.0],
            [np.sin(np.radians(angle)), np.cos(np.radians(angle)), 0.0],
            [0.0, 0.0, 1.0],
        ]
    )

In [ ]:
azel_correction = np.zeros((2, len(brightest_source_centroid)))

for i, source_xy in enumerate(brightest_source_centroid):
    dx_arcsec, dy_arcsec = calculate_xy_offsets(
        PointD(
            source_xy.brightestObjCentroid[0],
            source_xy.brightestObjCentroid[1]
        ), 
        boresight)

    # We are using rotator 2 so we must apply a negative sign on the x-axis offset.
    # The equation bellow return offset in elevation/azimuth.
    elaz_offset = np.matmul((-dx_arcsec, dy_arcsec, 0.), rotation_matrix(source_xy.angle))*u.arcsec
    
    elaz_offset[0] += source_xy.aos_offset["x"]*50.468*u.arcsec  # elevation
    elaz_offset[1] -= source_xy.aos_offset["y"]*52.459*u.arcsec  # azimuth
    
    # We want to store the offset in azel format, so we reverse the result given above.
    # The following was verified with the pointing component. When we add an offset of 
    # X arcsec in azimuth it results in a negative offset in the axis. When we make a
    # positive offset in elevation is results in a positive offset in the axis. The 
    # pointing takes care of the cos(elevation) dependency when we apply the offset, but
    # we need to take care of it here since we want to apply a correction to the axis directly.    
    azel_correction[0][i] = elaz_offset[1].to(u.deg).value * -1. / np.cos(np.radians(source_xy.elevation))
    azel_correction[1][i] = elaz_offset[0].to(u.deg).value

In [ ]:
plt.scatter(azel_correction[0], azel_correction[1])

## Apply correction to pointing data

Now that the corrections offsets are computed in az/el, we need to apply the offset to the appropriate columns and construct the pointing data. 

In [ ]:
pointing_data = np.array(
    [
        tuple(
            [
                data.point_data[key] for key in ("expectedAzimuth", "expectedElevation", "measuredAzimuth", "measuredElevation", "measuredRotator")
            ]
        )
        for data in brightest_source_centroid
    ],
    dtype = [(key, float) for key in ("expectedAzimuth", "expectedElevation", "measuredAzimuth", "measuredElevation", "measuredRotator")]
)

In [ ]:
corrected_pointing_data = pointing_data.copy()
corrected_pointing_data["measuredAzimuth"] += azel_correction[0]
corrected_pointing_data["measuredElevation"] += azel_correction[1]

In [ ]:
out_pointing_file, ext = os.path.splitext(pointing_data_file)
print(out_pointing_file, ext)

In [ ]:
now = datetime.now()

In [ ]:
now.second

In [ ]:
header = f"""LSST Auxiliary Telescope, {now.year} {now.month} {now.day} UTC {now.hour} {now.minute} {now.second}
: ALTAZ
: ROTNL
-30 14 40.3
"""
tail = "END"

In [ ]:
raw_filename = f"{out_pointing_file}.dat"
corrected_filename = f"{out_pointing_file}_corr.dat"

In [ ]:
with open(raw_filename, "w") as fp:
    print(f"Writting raw data to {raw_filename}.")
    fp.write(header)
    np.savetxt(fp, pointing_data, fmt="%011.7f %010.7f %011.7f %010.7f %011.7f")
    fp.write(tail)

with open(corrected_filename, "w") as fp:
    print(f"Writting corrected data to {corrected_filename}.")
    fp.write(header)
    np.savetxt(fp, corrected_pointing_data, fmt="%011.7f %010.7f %011.7f %010.7f %011.7f")
    fp.write(tail)